In [1]:
import pandas as pd
import os
import csv
import re
import sys
import numpy as np
import logging
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

logging.getLogger('numba').setLevel(logging.WARNING)
logging.getLogger('matplotlib.font_manager').disabled = True
logging.getLogger('matplotlib.colorbar').disabled = True
logging.getLogger('matplotlib.pyplot').disabled = True

In [2]:
# Function to extract the numeric part of the filename (after the last underscore and before the file extension)
def extract_numeric_part(path):
    # This regex extracts the number after the last underscore and before the file extension (.flac)
    match = re.search(r'_(\d+)\.flac$', path)
    if match:
        return match.group(1)  # Return only the numeric part
    return None  # Return None if the pattern doesn't match

def extract_numeric_part_npy(path):
    # This regex extracts the number after the last underscore and before the file extension (.flac)
    match = re.search(r'_E_(\d+)_None\.npy$', path)
    if match:
        return match.group(1)  # Return only the numeric part
    return None  # Return None if the pattern doesn't match


def pred_and_labels_clean(file1_csv, file2_csv, feature):
    '''
    file1: probabilities list .csv
    file2: eval dataset .csv
    '''
    
    prediction_list = []
    label_list = []
    pred_class1_list = []
    
    # Step 1: Read file2 into a dictionary for quick look-up based on the numeric part of the file path
    file2_dict = {}
    with open(file2_csv, mode='r') as file2:
        reader = csv.DictReader(file2)
        for row in reader:
            numeric_file2 = extract_numeric_part(row['path'])
            if numeric_file2:
                file2_dict[numeric_file2] = row['label']

    # Step 2: Traverse file1 and check against file2 dictionary, using tqdm for progress tracking
    with open(file1_csv, mode='r') as file1:
        reader = csv.DictReader(file1)
        total_rows = sum(1 for _ in open(file1_csv)) - 1  # Calculate total rows for the progress bar (excluding header)
    
        file1.seek(0)  # Reset the reader position back to the start of the file after counting
        if feature == 'audio':
            for row in reader:
                # Extract the numeric part from file1's path
                file1_path = row['Filename']
                numeric_file1 = extract_numeric_part(file1_path)
        
                # Step 3: Check if the numeric part exists in the file2 dictionary
                if numeric_file1 and numeric_file1 in file2_dict:
                    pred_class_0 = float(row['Pred.class 0'])
                    pred_class_1 = float(row['Pred.class 1'])
            
                    # Step 4: Append prediction based on comparison
                    prediction_list.append(0 if pred_class_0 > pred_class_1 else 1)
            
                    # Step 5: Append the corresponding label from file2
                    label_list.append(int(file2_dict[numeric_file1]))
                
                    # Step 6:  Append Pred.class 1 value to the new list
                    pred_class1_list.append(pred_class_1)
        elif feature == 'spec':
            for row in reader:
                # Extract the numeric part from file1's path
                file1_path = row['Filename']
                numeric_file1 = extract_numeric_part_npy(file1_path)
        
                # Step 3: Check if the numeric part exists in the file2 dictionary
                if numeric_file1 and numeric_file1 in file2_dict:
                    pred_class_0 = float(row['Pred.class 0'])
                    pred_class_1 = float(row['Pred.class 1'])
            
                    # Step 4: Append prediction based on comparison
                    prediction_list.append(0 if pred_class_0 > pred_class_1 else 1)
            
                    # Step 5: Append the corresponding label from file2
                    label_list.append(int(file2_dict[numeric_file1]))
                
                    # Step 6:  Append Pred.class 1 value to the new list
                    pred_class1_list.append(pred_class_1)
    
    return prediction_list, label_list, pred_class1_list




def pred_and_labels_attack(file1_csv, file2_csv):
    
    def extract_numeric_part(path):
        # This regex captures the numeric part following 'LA_E_' and stops at the next underscore or period
        match = re.search(r'LA_E_(\d+)', path)
        if match:
            return match.group(1)  # Return only the numeric part
        return None  # Return None if the pattern doesn't match
    
    prediction_list = []
    label_list = []
    pred_class1_list = []
    
    # Step 1: Read file2 into a dictionary for quick look-up based on the numeric part of the file path
    file2_dict = {}
    with open(file2_csv, mode='r') as file2:
        reader = csv.DictReader(file2)
        for row in reader:
            numeric_file2 = extract_numeric_part(row['path'])
            if numeric_file2:
                file2_dict[numeric_file2] = row['label']

    # Step 2: Traverse file1 and check against file2 dictionary, using tqdm for progress tracking
    with open(file1_csv, mode='r') as file1:
        reader = csv.DictReader(file1)
        total_rows = sum(1 for _ in open(file1_csv)) - 1  # Calculate total rows for the progress bar (excluding header)
    
        file1.seek(0)  # Reset the reader position back to the start of the file after counting
        for row in reader:
            # Extract the numeric part from file1's path
            file1_path = row['Filename']
            numeric_file1 = extract_numeric_part(file1_path)
        
            # Step 3: Check if the numeric part exists in the file2 dictionary
            if numeric_file1 and numeric_file1 in file2_dict:
                pred_class_0 = float(row['Pred.class 0'])
                pred_class_1 = float(row['Pred.class 1'])
            
                # Step 4: Append prediction based on comparison
                prediction_list.append(0 if pred_class_0 > pred_class_1 else 1)
            
                # Step 5: Append the corresponding label from file2
                label_list.append(int(file2_dict[numeric_file1]))
                
                # Step 6:  Append Pred.class 1 value to the new list
                pred_class1_list.append(pred_class_1)

    
    return prediction_list, label_list, pred_class1_list

In [3]:
def compute_eer(y_true, y_scores):
    # Step 1: Compute the ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)

    # Step 2: Compute FNR (1 - TPR)
    fnr = 1 - tpr

    # Step 3: Find the threshold where FPR is closest to FNR
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]

    # Step 4: EER is the value of FPR (or FNR) at that threshold
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]

    return eer, eer_threshold


def ROC(attack, attack_model, epsilon, eval_model, model_version, dataset, type_of_spec, feature, q_1=None, q_2=None):
   
    epsilon_str = str(epsilon).replace('.', 'dot')
    
    script_dir = os.getcwd()
    probs_csv_c = f'probs_{eval_model}_{model_version}_clean_{dataset}_{type_of_spec}_{feature}.csv'
    if attack != 'Ensemble' and attack != 'Ensemble1D' and attack != 'Ensemble1D_RS' and attack != 'Ensemble1D_RaS':
        probs_csv_p = f'probs_{eval_model}_{model_version}_{attack}_{attack_model}_{dataset}_{epsilon_str}_{type_of_spec}_{feature}.csv'
    elif attack == 'Ensemble':
        probs_csv_p = f'probs_{eval_model}_{model_version}_Ensemble_{dataset}_{q_1}_{q_2}_{epsilon_str}_{type_of_spec}_{feature}.csv'
    elif attack == 'Ensemble1D':
        probs_csv_p = f'probs_{eval_model}_{model_version}_{attack}_{dataset}_{q_1}_{q_2}_{epsilon_str}_{type_of_spec}_{feature}.csv'
    elif attack == 'Ensemble1D_RS':
        probs_csv_p = f'probs_{eval_model}_{model_version}_{attack}_{dataset}_{q_1}_{q_2}_{epsilon_str}_{type_of_spec}_{feature}.csv'
    elif attack == 'Ensemble1D_RaS':
        probs_csv_p = f'probs_{eval_model}_{model_version}_{attack}_{dataset}_{q_1}_{q_2}_{epsilon_str}_{type_of_spec}_{feature}.csv'    
        
    #print(probs_csv_p)
    if dataset == '3s':
        eval_csv = os.path.join(os.path.dirname(script_dir), 'data', 'df_eval_19_3s.csv' )
    else:
        eval_csv = os.path.join(os.path.dirname(script_dir), 'data', 'df_eval_19.csv' )
    
    # get GT labels
    pred_list_c, labels_list_c, pred_1_c = pred_and_labels_clean(file1_csv=probs_csv_c, file2_csv=eval_csv, feature=feature)
    labels_c = np.array(labels_list_c)
    pred_1_c = np.array(pred_1_c)
    
    pred_list_p, labels_list_p, pred_1_p = pred_and_labels_attack(file1_csv=probs_csv_p, file2_csv=eval_csv)
    labels_p = np.array(labels_list_p)
    pred_1_p = np.array(pred_1_p)
    
    if len(pred_list_c) != len(pred_list_p):
        sys.exit(f'length of clean is {len(pred_list_c)}, length of pert is {len(pred_list_p)}')
    
    # CLEAN - compute ROC, AUC, EER
    fpr_c, tpr_c, _ = roc_curve(labels_c, pred_1_c)
    roc_auc_c = auc(fpr_c, tpr_c)
    eer_c, _ = compute_eer(labels_c, pred_1_c)
    
    # PERTURBED - compute ROC, AUC, EER
    fpr_p, tpr_p, _ = roc_curve(labels_p, pred_1_p)
    roc_auc_p = auc(fpr_p, tpr_p)
    eer_p, _ = compute_eer(labels_p, pred_1_p)
    
    #plot
    # plt.figure()
    # plt.plot(fpr_c, tpr_c, color='darkorange', lw=1.5, label='Clean (AUC={:.2f})'.format(roc_auc_c))
    # plt.plot(fpr_p, tpr_p, color='red', lw=1.5, label='Perturbed (AUC={:.2f})'.format(roc_auc_p))
    # plt.plot([0, 1], [0, 1], color='navy', lw=1.5, linestyle='--')
    # # Set axis limits and labels
    # plt.xlim([0.0, 1.0])
    # plt.ylim([0.0, 1.05])
    # plt.xlabel('False Positive Rate')
    # plt.ylabel('True Positive Rate')
    # plt.title(f'ROC - {attack} on {attack_model}, eps={epsilon}, eval model: {eval_model}')
    # plt.legend(loc='lower right')
    # plt.show()
    print(f'Feature: {feature} - Eval model: {eval_model},\n attack: {attack}, attack model: {attack_model}, epsilon: {epsilon}\nAUC clean: {roc_auc_c:.3f}, EER clean: {eer_c*100:.3f}\nAUC perturbed: {roc_auc_p:.3f}, EER perturbed: {eer_p*100:.3f} \n------------------------')
    return fpr_c, tpr_c, roc_auc_c, fpr_p, tpr_p, roc_auc_p, eer_c, eer_p

In [4]:
fpr_Res2D_c, tpr_Res2D_c, roc_auc_Res2D_c, fpr_Res2D_BIM, tpr_Res2D_BIM, roc_auc_Res2D_BIM, eer_Res2D_c, eer_Res2D_BIM = ROC(attack='BIM', attack_model='ResNet2D', epsilon=2.0, eval_model='ResNet2D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_Res2D_BIM, tpr_Res2D_BIM, roc_auc_Res2D_BIM, eer_Res2D_c, eer_Res2D_BIM = ROC(attack='BIM', attack_model='ResNet2D', epsilon=2.0, eval_model='ResNet2D', model_version='v0', dataset='whole', type_of_spec='pow', feature='spec', q_1=None, q_2=None)

_,_,_, fpr_Res2D_BIM_Res1D, tpr_Res2D_BIM_Res1D, roc_auc_Res2D_BIM_Res1D, _, eer_Res2D_BIM_Res1D = ROC(attack='BIM', attack_model='ResNet1D', epsilon=None, eval_model='ResNet2D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_Res2D_BIM_SEn1D, tpr_Res2D_BIM_SEn1D, roc_auc_Res2D_BIM_SEn1D, _, eer_Res2D_BIM_SEn1D = ROC(attack='BIM', attack_model='SENet1D', epsilon=None, eval_model='ResNet2D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_Res2D_BIM_SEN2D, tpr_Res2D_BIM_SEN2D, roc_auc_Res2D_BIM_SEN2D, _, eer_Res2D_BIM_SEN2D = ROC(attack='BIM', attack_model='SENet2D', epsilon=2.0, eval_model='ResNet2D', model_version='v0', dataset='whole', type_of_spec='pow', feature='spec', q_1=None, q_2=None)

Feature: audio - Eval model: ResNet2D,
 attack: BIM, attack model: ResNet2D, epsilon: 2.0
AUC clean: 0.954, EER clean: 9.667
AUC perturbed: 0.043, EER perturbed: 90.075 
------------------------
Feature: spec - Eval model: ResNet2D,
 attack: BIM, attack model: ResNet2D, epsilon: 2.0
AUC clean: 0.954, EER clean: 9.667
AUC perturbed: 0.000, EER perturbed: 100.000 
------------------------
Feature: audio - Eval model: ResNet2D,
 attack: BIM, attack model: ResNet1D, epsilon: None
AUC clean: 0.954, EER clean: 9.667
AUC perturbed: 0.000, EER perturbed: 100.000 
------------------------
Feature: audio - Eval model: ResNet2D,
 attack: BIM, attack model: SENet1D, epsilon: None
AUC clean: 0.954, EER clean: 9.667
AUC perturbed: 0.852, EER perturbed: 21.849 
------------------------
Feature: spec - Eval model: ResNet2D,
 attack: BIM, attack model: SENet2D, epsilon: 2.0
AUC clean: 0.954, EER clean: 9.667
AUC perturbed: 0.947, EER perturbed: 10.537 
------------------------


In [5]:
# ResNet1D

fpr_Res1D_c, tpr_Res1D_c, roc_auc_Res1D_c, fpr_Res1D_BIM, tpr_Res1D_BIM, roc_auc_Res1D_BIM, eer_Res1D_c, eer_Res1D_BIM = ROC(attack='BIM', attack_model='ResNet1D', epsilon=None, eval_model='ResNet1D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_Res1D_BIM_Res2D, tpr_Res1D_BIM_Res2D, roc_auc_Res1D_BIM_Res2D, _, eer_Res1D_BIM_Res2D = ROC(attack='BIM', attack_model='ResNet2D', epsilon=2.0, eval_model='ResNet1D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_Res1D_BIM_SEn1D, tpr_Res1D_BIM_SEn1D, roc_auc_Res1D_BIM_SEn1D, _, eer_Res1D_BIM_SEn1D = ROC(attack='BIM', attack_model='SENet1D', epsilon=None, eval_model='ResNet1D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_Res1D_BIM_SEn2D, tpr_Res1D_BIM_SEn2D, roc_auc_Res1D_BIM_SEn2D, _, eer_Res1D_BIM_SEn2D = ROC(attack='BIM', attack_model='SENet2D', epsilon=2.0, eval_model='ResNet1D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

Feature: audio - Eval model: ResNet1D,
 attack: BIM, attack model: ResNet1D, epsilon: None
AUC clean: 0.968, EER clean: 7.179
AUC perturbed: 0.000, EER perturbed: 100.000 
------------------------
Feature: audio - Eval model: ResNet1D,
 attack: BIM, attack model: ResNet2D, epsilon: 2.0
AUC clean: 0.968, EER clean: 7.179
AUC perturbed: 0.762, EER perturbed: 24.174 
------------------------
Feature: audio - Eval model: ResNet1D,
 attack: BIM, attack model: SENet1D, epsilon: None
AUC clean: 0.968, EER clean: 7.179
AUC perturbed: 0.867, EER perturbed: 20.789 
------------------------
Feature: audio - Eval model: ResNet1D,
 attack: BIM, attack model: SENet2D, epsilon: 2.0
AUC clean: 0.968, EER clean: 7.179
AUC perturbed: 0.948, EER perturbed: 10.972 
------------------------


In [6]:
# SENet1D

fpr_SEN1D_c, tpr_SEN1D_c, roc_auc_SEN1D_c, fpr_SEN1D_BIM, tpr_SEN1D_BIM, roc_auc_SEN1D_BIM, eer_SEN1D_c, eer_SEN1D_BIM = ROC(attack='BIM', attack_model='SENet1D', epsilon=None, eval_model='SENet1D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_SEN1D_BIM_Res2D, tpr_SEN1D_BIM_Res2D, roc_auc_SEN1D_BIM_Res2D, _, eer_SEN1D_BIM_Res2D = ROC(attack='BIM', attack_model='ResNet2D', epsilon=2.0, eval_model='SENet1D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_SEN1D_BIM_Res1D, tpr_SEN1D_BIM_Res1D, roc_auc_SEN1D_BIM_Res1D, _, eer_SEN1D_BIM_Res1D = ROC(attack='BIM', attack_model='ResNet1D', epsilon=None, eval_model='SENet1D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_SEN1D_BIM_SEN2D, tpr_SEN1D_BIM_SEN2D, roc_auc_SEN1D_BIM_SEN2D, _, eer_SEN1D_BIM_SEN2D = ROC(attack='BIM', attack_model='SENet2D', epsilon=2.0, eval_model='SENet1D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

Feature: audio - Eval model: SENet1D,
 attack: BIM, attack model: SENet1D, epsilon: None
AUC clean: 0.947, EER clean: 10.374
AUC perturbed: 0.007, EER perturbed: 95.527 
------------------------
Feature: audio - Eval model: SENet1D,
 attack: BIM, attack model: ResNet2D, epsilon: 2.0
AUC clean: 0.947, EER clean: 10.374
AUC perturbed: 0.907, EER perturbed: 15.037 
------------------------
Feature: audio - Eval model: SENet1D,
 attack: BIM, attack model: ResNet1D, epsilon: None
AUC clean: 0.947, EER clean: 10.374
AUC perturbed: 0.223, EER perturbed: 69.082 
------------------------
Feature: audio - Eval model: SENet1D,
 attack: BIM, attack model: SENet2D, epsilon: 2.0
AUC clean: 0.947, EER clean: 10.374
AUC perturbed: 0.924, EER perturbed: 12.903 
------------------------


In [13]:
# SENet2D

fpr_SEN2D_c, tpr_SEN2D_c, roc_auc_SEN2D_c, fpr_SEN2D_BIM, tpr_SEN2D_BIM, roc_auc_SEN2D_BIM, eer_SEN2D_c, eer_SEN2D_BIM = ROC(attack='BIM', attack_model='SENet2D', epsilon=2.0, eval_model='SENet2D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_SEN2D_BIM_Res2D, tpr_SEN2D_BIM_Res2D, roc_auc_SEN2D_BIM_Res2D, _, eer_SEN2D_BIM_Res2D = ROC(attack='BIM', attack_model='ResNet2D', epsilon=2.0, eval_model='SENet2D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_SEN2D_BIM_Res1D, tpr_SEN2D_BIM_Res1D, roc_auc_SEN2D_BIM_Res1D, _, eer_SEN2D_BIM_Res1D = ROC(attack='BIM', attack_model='ResNet1D', epsilon=None, eval_model='SENet2D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

_,_,_, fpr_SEN2D_BIM_SEN1D, tpr_SEN2D_BIM_SEN1D, roc_auc_SEN2D_BIM_SEN1D, _, eer_SEN2D_BIM_SEN1D = ROC(attack='BIM', attack_model='SENet1D', epsilon=None, eval_model='SENet2D', model_version='v0', dataset='whole', type_of_spec='pow', feature='audio', q_1=None, q_2=None)

Feature: audio - Eval model: SENet2D,
 attack: BIM, attack model: SENet2D, epsilon: 2.0
AUC clean: 0.966, EER clean: 8.158
AUC perturbed: 0.317, EER perturbed: 62.651 
------------------------
Feature: audio - Eval model: SENet2D,
 attack: BIM, attack model: ResNet2D, epsilon: 2.0
AUC clean: 0.966, EER clean: 8.158
AUC perturbed: 0.920, EER perturbed: 13.705 
------------------------
Feature: audio - Eval model: SENet2D,
 attack: BIM, attack model: ResNet1D, epsilon: None
AUC clean: 0.966, EER clean: 8.158
AUC perturbed: 0.443, EER perturbed: 54.643 
------------------------
Feature: audio - Eval model: SENet2D,
 attack: BIM, attack model: SENet1D, epsilon: None
AUC clean: 0.966, EER clean: 8.158
AUC perturbed: 0.798, EER perturbed: 26.757 
------------------------
